
## Sliderule 3DEP sampling
### From Scott's Arctic DEM sampling

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import boto3
from sliderule import sliderule, icesat2

In [2]:
aoi_url = '../../sites/grandmesa.geojson'
gfa = gpd.read_file(aoi_url)
gfa.explore(style_kwds=dict(fill=False, color='magenta'))

In [3]:
icesat2.init("slideruleearth.io", verbose=True)

In [4]:
# Start with all defaults, just Polygon
parms = {
    "poly": sliderule.toregion(gfa)["poly"],
    # "srt": icesat2.SRT_LAND,
    # "cnf": icesat2.CNF_SURFACE_HIGH,
    # "len": 40.0,
    # "res": 20.0,
    # "maxi": 6
    "t0": "2023-01-01T00:00:00Z",
    "t1": "2024-01-01T00:00:00Z",
}

In [5]:
# Add Raster Sampling
# https://www.slideruleearth.io/web/rtd/user_guide/GeoRaster.html
parms["samples"] = {
            # "hls": {"asset": "landsat-hls", "bands": ["NDSI"]},
            "strip": {"asset": "usgs3dep-1meter-dem", "radius":10.0, "zonal_stats":True},
            #"mosaic": {"asset": "arcticdem-mosaic", "algorithm":"NearestNeighbour"},
            #"mosaic": {"asset": "arcticdem-mosaic", "radius": 10.0, "zonal_stats": True},
            #"strips": {"asset": "arcticdem-strips", "algorithm": "CubicSpline"}
        }

### Run sliderule

In [6]:
parms

{'poly': [{'lon': -108.15996262442047, 'lat': 39.00669498695481},
  {'lon': -108.12553849524684, 'lat': 39.00828408047391},
  {'lon': -108.12826515894373, 'lat': 39.03979372594071},
  {'lon': -108.21688172909325, 'lat': 39.0847831386408},
  {'lon': -108.23562754200954, 'lat': 39.06176272688245},
  {'lon': -108.2424442012517, 'lat': 39.05249952359773},
  {'lon': -108.19506841951798, 'lat': 39.01199182651902},
  {'lon': -108.15996262442047, 'lat': 39.00669498695481}],
 't0': '2023-01-01T00:00:00Z',
 't1': '2024-01-01T00:00:00Z',
 'samples': {'strip': {'asset': 'usgs3dep-1meter-dem',
   'radius': 10.0,
   'zonal_stats': True}}}

In [7]:
%%time

# Run SlideRule

gf = icesat2.atl06p(parms, version='006')

print("Start:", gf.index.min().strftime('%Y-%m-%d'))
print("End:", gf.index.max().strftime('%Y-%m-%d'))
print("Reference Ground Tracks: {}".format(gf["rgt"].unique()))
print("Cycles: {}".format(gf["cycle"].unique()))
print("Elevation Measurements: {} ".format(gf.shape[0]))
gf.head(2)

INFO:sliderule.earthdata:Identified 2 resources to process
CRITICAL:sliderule.sliderule:request <AppServer.144> failed to create raster strip for ATL03_20230206192127_07371802_006_01.h5
INFO:sliderule.sliderule:request <AppServer.144> processing initialized on ATL03_20230206192127_07371802_006_01.h5 ...
CRITICAL:sliderule.sliderule:request <AppServer.131> failed to create raster strip for ATL03_20230306061322_11561806_006_01.h5
INFO:sliderule.sliderule:request <AppServer.131> processing initialized on ATL03_20230306061322_11561806_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.131> processing of ATL03_20230306061322_11561806_006_01.h5 complete (0/0/0)
INFO:sliderule.sliderule:request <AppServer.131> processing complete (0/0/0/0)
INFO:sliderule.sliderule:request <AppServer.144> processing of ATL03_20230206192127_07371802_006_01.h5 complete (228548/38/0)
INFO:sliderule.sliderule:Successfully completed processing resource [2 out of 2]: ATL03_20230306061322_11561806_006_01.h5
IN

Start: 2023-02-06
End: 2023-02-06
Reference Ground Tracks: [737]
Cycles: [18]
Elevation Measurements: 284 
CPU times: user 102 ms, sys: 5.3 ms, total: 107 ms
Wall time: 7.79 s


,segment_id,distance,dh_fit_dx,rms_misfit,n_fit_photons,h_mean,rgt,pflags,h_sigma,w_surface_window_final,spot,cycle,gt,dh_fit_dy,geometry
time,,,,,,,,,,,,,,,
2023-02-06 19:24:34.552389888,216445,4.340650e+06,-0.221149,1.136813,13,3098.086954,737,0,0.359040,5.672367,6,18,10,0.0,POINT (-108.14366 39.00761)
2023-02-06 19:24:34.555207936,216446,4.340670e+06,0.644488,4.249098,66,3100.922838,737,0,0.531968,30.000000,6,18,10,0.0,POINT (-108.14368 39.00779)
